In [1]:
import scipy.io
import scipy.stats
import matplotlib.pyplot as plt
import re
import os
from IPython.display import Image 
import cv2
from tempfile import TemporaryFile
from scipy import stats
from torchvision import transforms
import pandas as pd
import shutil
import datetime
import glob
import torch.nn.functional as F
from scipy import integrate
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
from PIL import ImageFile, Image
import sys
sys.path.insert(1, '../architecture')
sys.path.insert(2, '../dataLoaders')
from ProUNet import *
import torchvision.models as models
from MapillaryDataLoader import *
from VolvoDataLoader import *
from MapillaryIntendedObjs import *
from tqdm import tqdm
from torcheval.metrics import R2Score
from torchsummary import summary
import torcheval
from torchmetrics.classification import MulticlassCalibrationError
%matplotlib inline

In [2]:
class MyMSE(torch.nn.Module):
    def __init__(self):
        super(MyMSE, self).__init__()

        
    def forward(self, output, target):
        
        mask = target != 0
        loss = target-output
        loss *= loss
        loss = loss*mask
        loss_mean = loss.sum()/(mask.sum()+1e-15)

        return loss_mean  

In [3]:
transform_in = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4191, 0.4586, 0.4700], [0.2553, 0.2675, 0.2945]),
    transforms.Resize((256,256))
])
transform_ou = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256,256))
])

In [4]:
args = {
    "mode":"val",
    "input_img_dim":(256,256),
    "mapillaryRootPath":'../../../datasets/augmented_mapillary',
    "cityscapesRootPath":'../../../datasets/augmented_cityscapes',
    "RSCDRootPath":'../../../datasets/RSCD',
    "volvoRootPath":"../../../datasets/augmented_volvo",
    "reducedCategories":True
}

dataloader = volvoDataLoader(**args)
val_loader = DataLoader(dataset = dataloader, batch_size = 32, shuffle = True, drop_last = True)

In [5]:
device = torch.device('cuda:0')
model = ProUNet(gecoConfig = {"enable":False}, device = device, num_classes = dataloader.get_num_classes(), LatentVarSize = 6, beta = 3, training = True, num_samples = 16).to(device)
check = torch.load("../checkpoints/HVAEFri_NewLossTest2/best.pth", map_location=device)
model.load_state_dict(check['model_state_dict'])

/home/lunet/wsmo6/.conda/envs/3.8/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


<All keys matched successfully>

In [6]:
mse = MyMSE()

In [7]:
loss_fri = {
"void":0,
"asphalt":0.,
"sky":0,
"snow":0.,
"ice":0.,
"vegetation":0,
"gravel":0.,
"wetAsphalt":0.,
}
r2 = 0
loss = 0

for i,batch in enumerate(val_loader):
    
    batchImg = batch['image'].to(device)
    FriLabel = batch['FriLabel'].to(device)

    _, _, frics = model.inference(batchImg)
    fric = torch.mean(frics, 0)
    loss += mse(output = fric, target = FriLabel).item()
            
    for value,className in enumerate(friClass):
        lossCat = 0
        FriLabelCat = torch.where(FriLabel == friClass[className], friClass[className], 0)
        loss_fri[className] += mse(output = fric, target = FriLabelCat).item()

    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [8]:
RMSE = {i[0]:(np.sqrt(i[1]/len(val_loader)), i[1]/len(val_loader)) for i in loss_fri.items() if i[0] not in ["sky", "vegetation", "void"]}
print("**************RMSE *********************** MSE")
RMSE

**************RMSE *********************** MSE


{'asphalt': (0.06961730705596539, 0.004846569441724569),
 'snow': (0.06590277003396235, 0.004343175098149325),
 'ice': (0.11778007551670082, 0.013872146188719747),
 'gravel': (0.0256055169930105, 0.0006556425004793495),
 'wetAsphalt': (0.03848842597851225, 0.0014813589343034169)}

In [9]:
mMSE_filter = np.mean([errors[1] for name, errors in RMSE.items() if name not in ["sky", "vegetation", "void"]])
print("Mean friction MSE over only surface textures:{}".format(loss/len(val_loader)))

Mean friction MSE over only surface textures:0.003996751171892837


In [10]:
mRMSE_filter = np.mean([errors[0] for name, errors in RMSE.items() if name not in ["sky", "vegetation", "void"]])
print("Mean friction RMSE over only surface textures:{}".format(np.sqrt(loss/len(val_loader))))

Mean friction RMSE over only surface textures:0.06321986374465574


# Results

### Segfri2/best.pth

1. asphalt: RMSE : 0.171, MSE : 0.029
2. snow: RMSE : 0.082, MSE : 0.007
3. ice: RMSE : 0.097, MSE : 0.009
4. gravel: RMSE : 0.146, MSE : 0.021
5. wetAsphalt: RMSE : 0.090, MSE : 0.008

- Mean friction MSE over only surface textures:0.0151
- Mean friction RMSE over only surface textures:0.123

### Segfri2/iterative.pth

1. asphalt: RMSE : 0.172, MSE : 0.029
2. snow: RMSE : 0.078, MSE : 0.006
3. ice: RMSE: 0.108, MSE: 0.0117
4. gravel: RMSE: 0.148, MSE : 0.021,
5. wetAsphalt: RMSE: 0.09, MSE: 0.009
            
- Mean friction MSE over only surface textures:0.014
- Mean friction RMSE over only surface textures:0.12


### RSCD+volvo/best.pth

1. asphalt: RMSE : 0.191, MSE : 0.0367
2. snow: RMSE : 0.1025, MSE : 0.0105
3. ice: RMSE : 0.126, MSE : 0.016
4. gravel: RMSE : 0.17, MSE : 0.029
5. wetAsphalt: RMSE : 0.09, MSE : 0.008
            
- Mean friction MSE over only surface textures:0.0196
- Mean friction RMSE over only surface textures:0.14
        

### RSCD+volvo/iterative.pth

1. asphalt: RMSE : 0.188, MSE : 0.035
2. snow: RMSE : 0.1, MSE : 0.01
3. ice: RMSE : 0.119, MSE : 0.014
4. gravel: RMSE : 0.166, MSE : 0.027
5. wetAsphalt: RMSE : 0.087, MSE : 0.007

- Mean friction MSE over only surface textures:0.019
- Mean friction RMSE over only surface textures:0.138

### NewLossTest2/best.pth

1. asphalt: RMSE : 0.0899, MSE : 0.008
2. snow: RMSE : 0.075, MSE : 0.006
3. ice: RMSE : 0.121, MSE : 0.015
4. gravel: RMSE : 0.042, 0.002
5. wetAsphalt: RMSE : 0.068, MSE : 0.005

- Mean friction MSE over only surface textures:0.006
- Mean friction RMSE over only surface textures:0.079

### NewLossTest2/iterative.pt

1. asphalt: RMSE : 0.141, MSE : 0.019
2. snow: RMSE : 0.088, MSE : 0.008
3. ice: RMSE : 0.112, MSE : 0.012
4. gravel: RMSE : 0.067, MSE : 0.004
5. wetAsphalt: RMSE : 0.084, MSE : 0.007
            
- Mean friction MSE over only surface textures:0.0108
- Mean friction RMSE over only surface textures:0.104